In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:\\Users\\eobuobi\\trial\\nlp_project\\data\\final\\smsspamcollection.tsv', sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df['message']

In [8]:
y = df['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vect = CountVectorizer()

In [12]:
# FIT VECTORIZER TO THE DATA(build a vocab, count thr number of words...)
count_vect.fit(X_train)
X_train_counts = count_vect.transform(X_train)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [14]:
# TRANSFORM THE ORIGINAL TEXT MESSAGE --> VECTOR
X_train_counts = count_vect.fit_transform(X_train)

In [16]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [18]:
X_train_counts.shape

(3733, 7082)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf_transformer = TfidfTransformer()

In [21]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [22]:
X_train_tfidf.shape

(3733, 7082)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
vectorizer = TfidfVectorizer()

In [25]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [26]:
from sklearn.svm import LinearSVC

In [27]:
clf = LinearSVC()

In [28]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

In [29]:
from sklearn.pipeline import Pipeline

In [30]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])

In [31]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [32]:
predictions = text_clf.predict(X_test)

In [33]:
X_test

3245    Squeeeeeze!! This is christmas hug.. If u lik ...
944     And also I've sorta blown him off a couple tim...
1044    Mmm thats better now i got a roast down me! i...
2484        Mm have some kanji dont eat anything heavy ok
812     So there's a ring that comes with the guys cos...
                              ...                        
4944    Check mail.i have mailed varma and kept copy t...
3313    I know you are serving. I mean what are you do...
3652         Want to send me a virtual hug?... I need one
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
4758    hey, looks like I was wrong and one of the kap...
Name: message, Length: 1839, dtype: object

In [34]:
from sklearn.metrics import confusion_matrix,classification_report

In [35]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [36]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [37]:
from sklearn import metrics

In [38]:
print(metrics.accuracy_score(y_test, predictions))

0.989668297988037


In [39]:
text_clf.predict(["Hi how are you doing today?"])

array(['ham'], dtype=object)

In [40]:
text_clf.predict(["Congratulations! You've been selected as a winner. TEXT WON to 44255 today for free contest"])

array(['spam'], dtype=object)